In [2]:
import cv2
import yaml
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

import yaml

In [4]:
yaml_filename = "/home/kodogyu/kimera_multi_ws/src/kimera_vio/params/L515_stereo/FrontendParams.yaml"
left_camera_filename = "/home/kodogyu/kimera_multi_ws/src/kimera_vio/params/L515_stereo/LeftCameraParams.yaml"
right_camera_filename = "/home/kodogyu/kimera_multi_ws/src/kimera_vio/params/L515_stereo/RightCameraParams.yaml"

with open(yaml_filename, "r") as stream:
    stream.readline()  # %YAML:1.0 건너뛰기
    yaml_data = yaml.safe_load(stream)
print("yaml loaded")

with open(left_camera_filename, "r") as l_stream:
    l_stream.readline()  # %YAML:1.0 건너뛰기
    left_camera_params = yaml.safe_load(l_stream)
print("left camera parameters loaded")

with open(right_camera_filename, "r") as r_stream:
    r_stream.readline()  # %YAML:1.0 건너뛰기
    right_camera_params = yaml.safe_load(r_stream)
print("right camera parameters loaded")


yaml loaded
left camera parameters loaded
right camera parameters loaded


In [48]:
# calculate Left camera -> Right camera transformation matrix
body_T_Lcam = np.array(left_camera_params["T_BS"]["data"]).reshape((4, 4))
body_T_Rcam = np.array(right_camera_params["T_BS"]["data"]).reshape((4, 4))
# print("body_T_Lcam: ")
# print(body_T_Lcam)
# print("body_T_Rcam: ")
# print(body_T_Rcam)

l_rotation = body_T_Lcam[:-1, :-1]
l_translation = body_T_Lcam[:-1, 3].reshape((3, 1))

# print("l_rotation: ")
# print(l_rotation)
# print("l_translation: ")
# print(l_translation)

l_inverse_rotation = l_rotation.transpose()
l_inverse_translation = -np.matmul(l_inverse_rotation, l_translation)
Lcam_T_body = np.column_stack([l_inverse_rotation, l_inverse_translation])
Lcam_T_body = np.row_stack([Lcam_T_body, np.array([0, 0, 0, 1])])

camL_Pose_camR = np.matmul(Lcam_T_body, body_T_Rcam)
print("camL_Pose_camR: ")
print(camL_Pose_camR)

camL_Pose_camR: 
[[ 9.99987688e-01 -2.47748079e-04  4.95770871e-03 -8.88571289e-03]
 [ 2.46216489e-04  9.99999921e-01  3.06972131e-04 -1.72242022e-04]
 [-4.95778223e-03 -3.05755384e-04  9.99987661e-01  1.97129796e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [66]:
l_fx, l_fy, l_cx, l_cy = left_camera_params["intrinsics"]
r_fx, r_fy, r_cx, r_cy = right_camera_params["intrinsics"]
left_camera_intrinsic = np.array([[l_fx, 0, l_cx], [0, l_fy, l_cy], [0, 0, 1]])
right_camera_intrinsic = np.array([[r_fx, 0, r_cx], [0, r_fy, r_cy], [0, 0, 1]])

left_camera_distCoeffs = np.array(left_camera_params["distortion_coefficients"])
right_camera_distCoeffs = np.array(right_camera_params["distortion_coefficients"])

camL_Rot_camR = camL_Pose_camR[:-1, :-1]
camL_Tran_camR = camL_Pose_camR[:-1, 3]
kAlpha = 0

r1, r2, p1, p2, q, roi1, roi2= cv2.stereoRectify(left_camera_intrinsic, 
                                                 left_camera_distCoeffs, 
                                                 right_camera_intrinsic, 
                                                 right_camera_distCoeffs, 
                                                 np.array(left_camera_params["resolution"]), 
                                                 camL_Rot_camR, 
                                                 camL_Tran_camR, 
                                                 flags=cv2.CALIB_ZERO_DISPARITY, 
                                                 alpha=kAlpha
                                                 )

stereo_camera = p1
stereo_baseline = 1 / q[3, 2]
print("stereo_camera:")
print(stereo_camera)
print("stereo_baseline:")
print(stereo_baseline)

stereo_camera:
[[648.62802467   0.         336.90341949   0.        ]
 [  0.         648.62802467 261.60647964   0.        ]
 [  0.           0.           1.           0.        ]]
stereo_baseline:
0.008889568098128261
